<a href="https://colab.research.google.com/github/tklee-123/AI/blob/main/notebooks/training_vietnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:
# Authenticate
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
OUTPUT_DIR = '/content/training/'

In [4]:
import random

random.seed(0)

In [5]:
!wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
!unzip 'master.zip'

--2025-08-29 10:17:01--  https://github.com/ThanhChinhBK/vietnews/archive/master.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ThanhChinhBK/vietnews/zip/refs/heads/master [following]
--2025-08-29 10:17:01--  https://codeload.github.com/ThanhChinhBK/vietnews/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip.1’

master.zip.1            [        <=>         ]  61.64M  11.7MB/s               ^C
Archive:  master.zip
a20d9b7f5d37b95f39f28f08999092f9fa400983
replace vietnews-master/data/test_tokenized/000001.txt.seg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
%%capture

# !wget 'https://github.com/CLC-HCMUS/ViMs-Dataset/raw/master/ViMs.zip'
# !unzip 'ViMs.zip'

# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter)
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

In [6]:
import glob
import pandas as pd
import concurrent.futures
from datasets import *

## Processing data

In [7]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g')

In [8]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('/content/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('/content/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('/content/vietnews-master/data/test_tokenized/*')

In [9]:
def read_content(pathfile):
  """
  Input: Path of txt file
  Output: A dictionary has keys 'original' and 'summary'
  """
  with open(pathfile) as f:
    rows  = f.readlines()
    original = ' '.join(''.join(rows[4:]).split('\n'))
    summary = ' '.join(rows[2].split('\n'))

  return {'file' : pathfile,
            'original': original,
            'summary': summary}

In [10]:
read_content(train_paths[0])

{'file': '/content/vietnews-master/data/train_tokenized/102818.txt.seg',
 'original': 'Chiều 4/11 , ông Nguyễn_Thế_Hùng - Phó chủ_tịch UBND TP Hội_An , cho_biết do ảnh_hưởng của bão số 12 kết_hợp không_khí lạnh , nước_biển đã xâm_thực mạnh vào bờ biển Cửa_Đại , nhiều nơi sóng đánh cao khoảng 10 mét gây sạt_lở . Sóng đánh cao 10 mét ở Cửa_Đại ( Hội_An ) . Một_số tuyến phố Hội_An ngập nước khoảng 30 cm , trong khi nước lũ đang lên nhanh . TP Hội_An ra thông_báo trên loa phát_thanh về việc thuỷ_điện sông Tranh 2 xả lũ vào chiều cùng ngày từ 500 đến 1.500 m3 / s . Từ 18h ngày 4/1 , thuỷ_điện sông Tranh 2 xả lũ 2.000 đến 3.500 m3 / s . Việc thuỷ_điện xả lũ kết_hợp mưa lớn khiến TP Hội_An xuất hơn đợt lũ được dự_báo trên báo_động báo ba từ ngày_mai , khả_năng đạt đỉnh lũ năm 2016 . “ Các cơ_quan , đơn_vị , xã phường triển_khai trực 24/24 để theo_dõi tình_hình ; chuẩn_bị đầy_đủ vật_tư , phương_tiện để chủ_động phòng_chống lũ ” , ông Hùng nói . TP Hội_An đã triển_khai phương_án di_dời tại_chỗ 

In [11]:
def get_dataframe(pathfiles):
  with concurrent.futures.ProcessPoolExecutor() as executor:
    data = executor.map(read_content, pathfiles)

  # Make blank dataframe
  data_df = list()
  for d in data:
    data_df.append(d)
  data_df = pd.DataFrame(data_df)
  data_df.dropna(inplace = True)
  data_df = data_df.sample(frac=1).reset_index(drop=True)

  return data_df

In [12]:
train_df = get_dataframe(train_paths)

In [13]:
type(train_df)

pandas.core.frame.DataFrame

In [14]:
val_df = get_dataframe(val_paths)
test_df = get_dataframe(test_paths)

## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [17]:
!pip install dill==0.3.5.1

  Using cached dill-0.3.5.1-py2.py3-none-any.whl.metadata (9.7 kB)
Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
evaluate 0.4.5 requires datasets>=2.0.0, but you have datasets 1.0.2 which is incompatible.
multiprocess 0.70.16 requires dill>=0.3.8, but you have dill 0.3.5.1 which is incompatible.


In [13]:
%%capture
!pip install datasets==1.0.2
!pip install transformers

import datasets
import transformers

In [17]:
from transformers import RobertaTokenizerFast,AutoTokenizer

# phobert = AutoModel.from_pretrained("vinai/phobert-base")

# For transformers v4.x+:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)


In [18]:
from sklearn.model_selection import train_test_split

train_data =  Dataset.from_pandas(train_df)
val_data =  Dataset.from_pandas(val_df)
test_data =  Dataset.from_pandas(test_df)

In [19]:
batch_size=16  # change to 16 for full training
encoder_max_length=256
decoder_max_length=64

def process_data_to_model_inputs(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    # mask loss for padding
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]
    batch["decoder_attention_mask"] = outputs.attention_mask

    return batch

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
# train_data = train_data.select(range(32))

train_data_batch = train_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["file","original", "summary"],
)
train_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data_batch = val_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["file", "original", "summary"],
)
val_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

  0%|          | 0/6589 [00:00<?, ?ba/s]

  0%|          | 0/1416 [00:00<?, ?ba/s]

### **Warm-starting the Encoder-Decoder Model**

In [20]:
from transformers import EncoderDecoderModel

# set encoder decoder tying to True
roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("vinai/phobert-base", "vinai/phobert-base", tie_encoder_decoder=True)

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['roberta.encoder.layer.0.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.0.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.0.crossattention.output.dense.bias', 'roberta.encoder.layer.0.crossattention.output.dense.weight', 'roberta.encoder.layer.0.crossattention.self.key.bias', 'roberta.encoder.layer.0.crossattention.self.key.weight', 'roberta.encoder.layer.0.crossattention.self.query.bias', 'roberta.encoder.layer.0.crossattention.self.query.weight', 'roberta.encoder.layer.0.crossattention.self.value.bias', 'roberta.encoder.layer.0.crossattention.self.value.weight', 'roberta.encoder.layer.1.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.1.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.1.crossattention.output.dense.bias', 'roberta.encoder.layer.1.crossattention.output.dense.weight', 'roberta.encod

In [21]:
# set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id
roberta_shared.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params
roberta_shared.config.max_length = 64
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

In [22]:
!rm seq2seq_trainer.py
!wget https://github.com/huggingface/transformers/blob/main/examples/legacy/seq2seq/seq2seq_trainer.py

!pip install git-python==1.0.3
!pip install rouge_score

--2025-08-29 10:30:31--  https://github.com/huggingface/transformers/blob/main/examples/legacy/seq2seq/seq2seq_trainer.py
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘seq2seq_trainer.py’

seq2seq_trainer.py      [  <=>               ] 216.76K   924KB/s    in 0.2s    

2025-08-29 10:30:32 (924 KB/s) - ‘seq2seq_trainer.py’ saved [221958]



In [29]:
!pip uninstall -y sacrebleu mecab-python3
!pip install "sacrebleu>=2.4,<3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
  Using cached portalocker-3.2.0-py3-none-any.whl.metadata (8.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.3 MB/s eta 0:00:00


In [31]:
!wget -O seq2seq_trainer.py \ https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_trainer.py

 https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_trainer.py: Scheme missing.


In [23]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, Trainer
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [24]:
import datasets

In [25]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

Cool! Finally, we start training.

In [26]:
!pip install -U datasets evaluate

  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
Using cached datasets-4.0.0-py3-none-any.whl (494 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1
  Attempting uninstall: datasets
    Found existing installation: datasets 1.0.2
    Uninstalling datasets-1.0.2:
      Successfully uninstalled datasets-1.0.2


In [29]:
import numpy as np
if not hasattr(np, "object"):
    np.object = object


/tmp/ipython-input-1078063041.py:2: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [32]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir= OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    # evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    logging_steps=200,  # set to 2000 for full training
    save_steps=5000,  # set to 500 for full training
    eval_steps=7500,  # set to 7500 for full training
    warmup_steps=3000,  # set to 3000 for full training
    num_train_epochs=10, #uncomment for full training
    overwrite_output_dir=True,
    save_total_limit=50,
    fp16=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data_batch,
    eval_dataset=val_data_batch,
)

# Train pre-trained model
trainer.train()

Step,Training Loss
200,5.200900
400,0.716300
600,0.053100
800,0.008900
1000,0.003500
1200,0.002800
1400,0.003300
1600,0.003000
1800,0.002000
2000,0.004200


Step,Training Loss
200,5.200900
400,0.716300
600,0.053100
800,0.008900
1000,0.003500
1200,0.002800
1400,0.003300
1600,0.003000
1800,0.002000
2000,0.004200


KeyboardInterrupt: 

In [ ]:
!gsutil -m cp -r '/content/training/*' 'gs://kaggle-vbdi-test/training_Data'

### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [ ]:
import datasets
from transformers import RobertaTokenizer, EncoderDecoderModel, AutoTokenizer
from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

model = EncoderDecoderModel.from_pretrained(OUTPUT_DIR + "/checkpoint-4000")
model.to("cuda")

# test_data = datasets.load_dataset("xsum", split="test")

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

pred_str = results["pred"]
label_str = results["summary"]

In [ ]:
rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1","rouge2","rougeL"])

In [ ]:
for key,value in rouge_output.items():
  print(key)
  print(value.mid)

rouge1
Score(precision=0.6156883807011324, recall=0.6143045549259318, fmeasure=0.5906751582344583)
rouge2
Score(precision=0.3081612284527864, recall=0.3055766169704671, fmeasure=0.29421411955015214)
rougeL
Score(precision=0.4081913955092156, recall=0.405197179436527, fmeasure=0.3907000488647073)


In [ ]:
i = 154
print('Prediction: ',pred_str[i])
print('Truth: ',label_str[i])
print('Content: ',test_data[i]['original'])

Prediction:  Tên trộm đã đột_nhập đền Xã_Tắc ( phường Ka_Long, TP. Móng_Cái ) lấy trộm khoảng 300 triệu đồng từ hòm công_đức.
Truth:  Camera tại đền Xã_Tắc ( Quảng_Ninh ) ghi lại hình_ảnh hai thanh_niên cạy cửa đền , dùng thanh sắt và đục phá hỏng 3 chiếc hòm công_đức lấy đi số tiền ước_tính 300 triệu đồng .
Content:  Công_an thành_phố Móng_Cái ( Quảng_Ninh ) đang truy_tìm hai nghi phạm trộm tài_sản tại đền Xã_Tắc ở phường Ka_Long . Trước đó , khoảng 2h15 ngày 28/5 , hai thanh_niên đột_nhập đền , cạy cửa , phá hỏng 3 chiếc hòm công_đức lấy đi toàn_bộ tiền bên trong . Tài_sản bị mất ước_chừng 300 triệu đồng , chủ_yếu có mệnh_giá 10.000-50.000 đồng . Hình_ảnh do camera an_ninh ghi lại cho thấy hai nghi phạm tuổi từ 25 đến 30 . Phân_tích các dữ_liệu , cảnh_sát nhận_định , một người cao khoảng 1m65 , miệng rộng , má hóp , dáng gầy , mặc áo_phông ngắn tay màu nâu , đội mũ lưỡi chai màu ghi . Người thứ hai cao khoảng 1m70 , dáng gầy , mặc quần vải màu đen , áo sơmi dài tay tối màu , đội mũ c

In [ ]:
test_data[i]['file']

'/content/ViMs/original/Cluster_256/original/1671.txt'

In [ ]:
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g')

In [ ]:
text = '''
 Ngày 4/2, lãnh đạo thị xã Hoàng Mai (Nghệ An) cho biết, cơ quan chức năng thị xã vừa phối hợp với đơn vị chức năng tỉnh cách ly, lấy mẫu xét nghiệm cho 1 nam sinh viên trường Đại học FPT có biểu hiện ho sốt sau khi đi từ Hà Nội về nhà.

Trước đó vào ngày 31/1 nam sinh C.T.Ph. (20 tuổi, trú TX. Hoàng Mai, Nghệ An; Sinh viên trường Đại học FPT cơ sở Mỹ Đình) cùng 1 người bạn đi xe máy từ Hà Nội về quê nhà.

Lúc 22h ngày 31/1 nam sinh này về đến quê nhà nhưng không khai báo y tế. Từ ngày 1-3/2, nam sinh Ph. có đi một số nơi trên địa bàn thị xã và tiếp xúc với khoảng 12 người.


Khoảng 20h ngày 3/2, nam sinh này có dấu hiệu sốt 39 độ, đau đầu, đau rát họng, đau mỏi cơ, khó thở nhẹ. Nam sinh này sau đó được đưa vào Bệnh viện Phong da liễu (TX. Hoàng Mai) để khám và cách ly y tế.

Nhận được tin báo, Trung tâm Kiểm soát bệnh tật tỉnh Nghệ An đã trực tiếp ra lấy mẫu bệnh phẩm để xét nghiệm Covid-19. Dự kiến trong chiều cùng ngày sẽ có kết quả.

Hiện cơ quan chức năng đang rà soát những người tiếp xúc gần với nam sinh Ph. để có phương án theo dõi, cách ly và lấy mẫu xét nghiệm.

Trước đó vào đêm 31/1, một chuyến xe khách chở 22 người từ Hà Nội về Nghệ An trong đó có nhiều sinh viên Đại học FPT đi về tại phường Lê Lợi, TP. Vinh (Nghệ An). Cơ quan chức năng sau đó đã đưa 1 trường hợp F1 trên xe khách này đi cách ly tại Trung tâm Y tế huyện Nam Đàn. Riêng những trường hợp còn lại được theo dõi tại nhà.

Sau khi cách ly, các trường hợp này đã được lấy mẫu xét nghiệm và cho kết quả âm tính với Covid-19.
 '''

In [ ]:
text = rdrsegmenter.tokenize(text)
text = ' '.join([' '.join(x) for x in text])
text

'Ngày 4/2 , lãnh_đạo thị_xã Hoàng_Mai ( Nghệ_An ) cho biết , cơ_quan_chức_năng thị_xã vừa phối_hợp với đơn_vị chức_năng tỉnh cách_ly , lấy mẫu xét_nghiệm cho 1 nam sinh_viên trường Đại_học FPT có biểu_hiện ho sốt sau khi đi từ Hà_Nội về nhà . Trước đó vào ngày 31/1 nam_sinh C.T.Ph. ( 20 tuổi , trú TX . Hoàng_Mai , Nghệ_An ; Sinh_viên trường Đại_học FPT cơ_sở Mỹ_Đình ) cùng 1 người bạn đi xe_máy từ Hà_Nội về quê nhà . Lúc 22h ngày 31/1 nam_sinh này về đến quê nhà nhưng không khai_báo y_tế . Từ ngày 1-3/2 , nam_sinh Ph . có đi một_số nơi trên địa_bàn thị_xã và tiếp_xúc với khoảng 12 người . Khoảng 20h ngày 3/2 , nam_sinh này có dấu_hiệu sốt 39 độ , đau_đầu , đau rát họng , đau mỏi cơ , khó thở nhẹ . Nam_sinh này sau đó được đưa vào Bệnh_viện Phong da_liễu ( TX . Hoàng_Mai ) để khám và cách_ly y_tế . Nhận được tin báo , Trung_tâm Kiểm_soát bệnh_tật tỉnh Nghệ_An đã trực_tiếp ra lấy mẫu bệnh_phẩm để xét_nghiệm Covid-19 . Dự_kiến trong chiều cùng ngày sẽ có kết_quả . Hiện cơ_quan_chức_năng đ

In [ ]:
inputs = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
input_ids = inputs.input_ids.to("cuda")
attention_mask = inputs.attention_mask.to("cuda")

outputs = model.generate(input_ids, attention_mask=attention_mask)

# all special tokens including will be removed
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
output_str

['Trong người bạn gái của một cán_bộ Công_an tỉnh Nghệ_An cho biết, đơn_vị đang khẩn_trương điều_tra, xác_minh nguyên_nhân khiến một nam thanh_niên tử_vong.']

In [20]:
!pip install --upgrade dill